In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from pyts.datasets import make_cylinder_bell_funnel

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
random_state = 0

# IMPORT DATASET

In [4]:
dataset_name = "cbf"

In [5]:
X_all, y_all = make_cylinder_bell_funnel(n_samples = 600, random_state = random_state)
X_all = X_all.reshape((X_all.shape[0], X_all.shape[1], 1))

In [6]:
print("X SHAPE: ", X_all.shape)
print("y SHAPE: ", y_all.shape)
unique, counts = np.unique(y_all, return_counts=True)
print("\nCLASSES BALANCE")
for i, label in enumerate(unique):
    print(label, ": ", round(counts[i]/sum(counts), 2))

X SHAPE:  (600, 128, 1)
y SHAPE:  (600,)

CLASSES BALANCE
0 :  0.33
1 :  0.33
2 :  0.33


## DATASET SPLIT

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# BLACKBOX/EXPLANATION SETS SPLIT
X_train, X_exp, y_train, y_exp = train_test_split(X_all, y_all, 
                                                  test_size=0.3, stratify = y_all, random_state=random_state)

# BLACKBOX TRAIN/TEST SETS SPLIT
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# BLACKBOX TRAIN/VALIDATION SETS SPLIT
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, stratify = y_train, random_state=random_state)

# EXPLANATION TRAIN/TEST SETS SPLIT
X_exp_train, X_exp_test, y_exp_train, y_exp_test = train_test_split(X_exp, y_exp, 
                                                                    test_size=0.2, 
                                                                    stratify = y_exp, 
                                                                    random_state=random_state)

# EXPLANATION TRAIN/VALIDATION SETS SPLIT
X_exp_train, X_exp_val, y_exp_train, y_exp_val = train_test_split(X_exp_train, y_exp_train, 
                                                                  test_size=0.2, 
                                                                  stratify = y_exp_train, 
                                                                  random_state=random_state)

print("SHAPES:")
print("BLACKBOX TRAINING SET: ", X_train.shape)
print("BLACKBOX VALIDATION SET: ", X_val.shape)
print("BLACKBOX TEST SET: ", X_test.shape)
print("EXPLANATION TRAINING SET: ", X_exp_train.shape)
print("EXPLANATION VALIDATION SET: ", X_exp_val.shape)
print("EXPLANATION TEST SET: ", X_exp_test.shape)

SHAPES:
BLACKBOX TRAINING SET:  (268, 128, 1)
BLACKBOX VALIDATION SET:  (68, 128, 1)
BLACKBOX TEST SET:  (84, 128, 1)
EXPLANATION TRAINING SET:  (115, 128, 1)
EXPLANATION VALIDATION SET:  (29, 128, 1)
EXPLANATION TEST SET:  (36, 128, 1)


In [9]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_all)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  128
N. LABELS:  3


# BLACKBOX MODEL

In [10]:
from joblib import load
import keras
"""import importlib
importlib.reload(blackboxes)"""
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import time
import blackboxes
from blackboxes import *

Using TensorFlow backend.
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/frances

## resnet

In [11]:
blackbox = build_resnet(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/cbf_blackbox_resnet_20191106_145242_best_weights_+1.00_.hdf5")
resnet = blackbox

Instructions for updating:
Colocations handled automatically by placer.


## simple CNN

In [12]:
blackbox = build_simple_CNN(n_timesteps, n_outputs)
blackbox.load_weights(parentdir + "/blackbox_checkpoints/cbf_blackbox_simpleCNN_20191106_145515_best_weights_+1.00_.hdf5")
simplecnn = blackbox

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, mean_squared_error, accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [14]:
knn = load(parentdir + "/blackbox_checkpoints/cbf_blackbox_knn_20191106_145654.joblib")

# AUTOENCODER

In [15]:
"""import importlib
importlib.reload(autoencoders)"""
import autoencoders
from autoencoders import *

## STANDARD

In [16]:
params = {"input_shape": (n_timesteps,1),
          "n_blocks": 8, 
          "latent_dim": 2,
          "encoder_latent_layer_type": "dense",
          "encoder_args": {"filters":[2,4,8,16,32,64,128,256], 
                            "kernel_size":[21,18,15,13,11,8,5,3], 
                            "padding":"same", 
                            "activation":"elu", 
                            "pooling":[1,1,1,1,1,1,1,1]}
         }

aut = Autoencoder(verbose = False, **params)
encoder, decoder, autoencoder = aut.build()
autoencoder.load_weights(parentdir+"/autoencoder_checkpoints/cbf_autoencoder_20191106_144056_best_weights_+1.0504_.hdf5")

# GLOBAL VS LOCAL SHAPELET TREE

In [17]:
import agnosticlocalexplainer
from agnosticlocalexplainer import *
import importlib
importlib.reload(agnosticlocalexplainer)

/Users/francesco/anaconda3/envs/tesi/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


<module 'agnosticlocalexplainer' from '/Users/francesco/github/TS_AgnosticLocalExplainer/agnosticlocalexplainer.py'>

In [18]:
"""import importlib
import global_vs_local_surrogate
importlib.reload(global_vs_local_surrogate)"""

'import importlib\nimport global_vs_local_surrogate\nimportlib.reload(global_vs_local_surrogate)'

In [19]:
from myutils import BlackboxPredictWrapper
import time
from agnosticglobalexplainer import AgnosticGlobalExplainer, save_shapelet_model, load_shapelet_model
from joblib import dump
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, coverage_error
from global_vs_local_surrogate import build_agnostic_local_explainers, print_report, massive_save_agnostic_local_explainers, massive_load_agnostic_local_explainers, get_all_predictions

## RESNET

In [20]:
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]
blackbox = resnet
blackbox_input_dimensions = 3
blackbox_predict = BlackboxPredictWrapper(blackbox, 3)
labels = ["cylinder", "bell", "funnel"]

In [26]:
%%time
file_path = parentdir + "/agnostic_explainers/cbf_knn_20200112_224957"

global_surrogate = load_shapelet_model(file_path + "/")
agnostic_explainers = massive_load_agnostic_local_explainers(file_path, verbose = True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
1 / 36
2 / 36
3 / 36
4 / 36
5 / 36
6 / 36
7 / 36
8 / 36
9 / 36
10 / 36
11 / 36
12 / 36
13 / 36
14 / 36
15 / 36
16 / 36
17 / 36
18 / 36
19 / 36
20 / 36
21 / 36
22 / 36
23 / 36
24 / 36
25 / 36
26 / 36
27 / 36
28 / 36
29 / 36
30 / 36
31 / 36
32 / 36
33 / 36
34 / 36
35 / 36
36 / 36


NameError: name 'Users' is not defined

In [ ]:
# sanity check
results_df_loaded = get_all_predictions(agnostic_explainers, global_surrogate, blackbox_predict, X_exp_test, blackbox_predict.predict(X_exp_train), encoder, decoder)
#print(sum(results_df_loaded.values != results_df.values))

In [ ]:
print_report(results_df)

In [87]:
import agnosticglobalexplainer
importlib.reload(agnosticglobalexplainer)

<module 'agnosticglobalexplainer' from '/Users/francesco/github/TS_AgnosticLocalExplainer/agnosticglobalexplainer.py'>

In [88]:
import global_vs_local_surrogate
importlib.reload(global_vs_local_surrogate)

from global_vs_local_surrogate import calculate_shapelet_stability, calculate_shapelet_multi_stability

In [61]:
prova=calculate_shapelet_stability(agnostic_explainers,X_exp_test, n_neighbors = 30)


far: [0.0109427  0.02188539 0.        ]
near: [0.11141746 0.22283492 0.        ]

far: [0.01071968 0.02143936 0.        ]
near: [0.04161891 0.08323782 0.        ]

far: [0.01376608 0.02753215 0.        ]
near: [0.18156533 0.36313067 0.        ]

far: [0. 0. 0.]
near: [0.02652036 0.05304072 0.        ]

far: [0. 0. 0.]
near: [0.02939751 0.05879501 0.        ]

far: [0. 0. 0.]
near: [0.66275808 1.         0.32551616]

far: [0. 0. 0.]
near: [0.79289149 1.         0.58578299]

far: [0.00828395 0.0165679  0.        ]
near: [0.52357313 1.         0.04714625]

far: [0.01088265 0.02176529 0.        ]
near: [0.19926362 0.20861892 0.18990833]

far: [0.01738912 0.03477825 0.        ]
near: [0.02652036 0.05304072 0.        ]

far: [0.00599387 0.01198775 0.        ]
near: [0.575716 1.       0.151432]

far: [0.01243919 0.02487838 0.        ]
near: [0.5229025  1.         0.04580501]

far: [0.00943029 0.01886058 0.        ]
near: [0.52353508 1.         0.04707015]

far: [0. 0. 0.]
near: [0.03475714 0

In [89]:
prova=calculate_shapelet_multi_stability(agnostic_explainers,X_exp_test)


debug 1
far: [0.08223561 0.16447123 0.        ]
debug 0
far: [0.1028923  0.19474703 0.01103758]
debug 1
far: [0.01846029 0.03692058 0.        ]
debug 1
far: [0.01650298 0.03300595 0.        ]
debug 0
far: [0.05849228 0.09716859 0.01981596]
debug 1
far: [0.01164814 0.02329629 0.        ]
debug 1
near: [0.11141746 0.22283492 0.        ]

debug 1
far: [0.02537388 0.05074776 0.        ]
debug 1
far: [0.02841326 0.05682653 0.        ]
debug 1
far: [0.09720886 0.19441773 0.        ]
debug 1
far: [0.0535187 0.1070374 0.       ]
debug 0
far: [0.01930022 0.02722525 0.0113752 ]
debug 1
far: [0.00548497 0.01096993 0.        ]
debug 1
near: [0.04161891 0.08323782 0.        ]

debug 1
far: [0.08414    0.16827999 0.        ]
debug 1
far: [0.05166086 0.10332172 0.        ]
debug 0
far: [0.51020648 1.         0.02041296]


In [63]:
np.mean(prova[:,0])

0.1293143850377449

In [42]:
from shap_utils import shap_stability

In [44]:
params = {"background": "linear_consecutive"}

In [45]:
shap_stability(X_exp_test, blackbox,blackbox_input_dimensions, **params)


far: 0.7505619441639249
near: 0.9107130756894721
near: 0.8987261359457019
near: 0.9017965478100903

far: 0.7875307629204266
near: 0.8809696784307727
near: 0.9768506741287204
near: 0.9768506741287204

far: 0.9378489021213249
near: 0.9457776804755473
near: 0.9379470776388487
near: 0.9405141881883674

far: 0.7499302520226914
near: 0.8747152619589978
near: 0.964622147791501
near: 0.8747152619589978

far: 0.7817627470274305
near: 0.9597600599850037
near: 0.9404575167209942
near: 0.9321788988174368

far: 0.790743282996666
near: 0.91144391070924
near: 0.9042893187552565
near: 0.9255136003672672

far: 0.8010569501425492
near: 0.9323944639078705
near: 0.9315866084425037
near: 0.8881350705419782

far: 0.7889330228735791
near: 0.8782161234991424
near: 0.927536231884058
near: 1.0

far: 0.7887401089603773
near: 0.9461346223791813
near: 0.9405946368376238
near: 0.9673201569016326

far: 0.7757575757575758
near: 0.8747152619589978
near: 0.9768506741287204
near: 0.8782161234991424

far: 0.801104697946

array([[1.21337497, 1.19740435, 1.20409149],
       [1.24039685, 1.11864796, 1.19981388],
       [1.00845422, 1.00010468, 1.00380027],
       [1.28628248, 1.16639549, 1.20635782],
       [1.22768713, 1.19240639, 1.20769653],
       [1.17043498, 1.14359406, 1.15555701],
       [1.16395528, 1.10870403, 1.14520203],
       [1.26753472, 1.11316943, 1.18546284],
       [1.22641177, 1.19252796, 1.20616385],
       [1.25922157, 1.12756264, 1.17295323],
       [1.17401243, 1.14797117, 1.15772375],
       [0.97022797, 0.93659674, 0.95249438],
       [1.28826422, 1.15317469, 1.19987485],
       [1.21808252, 1.17786731, 1.20190426],
       [1.22468262, 1.19348529, 1.21203363],
       [1.10147228, 1.04479039, 1.07615859],
       [1.09200438, 1.05081683, 1.07017266],
       [1.198691  , 1.17400932, 1.18753737],
       [1.22305442, 1.21178039, 1.21839973],
       [1.14908525, 1.07580175, 1.1050639 ],
       [0.99518338, 0.98586487, 0.99115975],
       [1.24139397, 1.13879142, 1.18100129],
       [1.